In [1]:
import xarray as xr
import glob
import numpy as np
import netCDF4 as nc
import warnings
import matplotlib.path as mpath
import cartopy.feature as cfeature

warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt
from cmocean import cm
import warnings
warnings.filterwarnings('ignore')
import cartopy.crs as ccrs
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/MO_pipeline/')
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS/')
import scendict as sc
import utils as ut

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [9]:
## seasonal cycles for each year for each latitude band (0-10) 

tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]

so_csize = (tmesh['csize'][:,:].values)
so_csize_broad = np.zeros([365,149, 182])

for i in range(0,365):
    so_csize_broad[i,:,:] = so_csize

print(np.shape(so_csize_broad))

(365, 149, 182)


In [12]:
tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
csize = tmesh['csize'].values

bounds = np.array([0,7,16,22,28,32,37,40,44,47,50])
starts = np.zeros([10]); ends = np.zeros([10])
print(' latitude bands:')
for i in range(0,len(bounds)-1):
    s= np.format_float_positional(tmesh['nav_lat'][bounds[i],0].values, precision=1)
    e= np.format_float_positional(tmesh['nav_lat'][bounds[i+1],0].values, precision=1)
    print(f'band {i}, {(s)} to {e}, indexes ({bounds[i]}-{bounds[i+1]})' )
    starts[i] = s
    ends[i] = e
    
print()
print('latitudes')
    
for i in [0, 36, 37, 49, 50]:
    tlat = tmesh['nav_lat'][i,0].values
    print(f'index {i}, latitude {tlat} N')

 latitude bands:
band 0, -78.2 to -75., indexes (0-7)
band 1, -75. to -69.5, indexes (7-16)
band 2, -69.5 to -64.9, indexes (16-22)
band 3, -64.9 to -59.3, indexes (22-28)
band 4, -59.3 to -54.9, indexes (28-32)
band 5, -54.9 to -48.8, indexes (32-37)
band 6, -48.8 to -44.6, indexes (37-40)
band 7, -44.6 to -38.7, indexes (40-44)
band 8, -38.7 to -33.8, indexes (44-47)
band 9, -33.8 to -28.7, indexes (47-50)

latitudes
index 0, latitude -78.19058227539062 N
index 36, latitude -50.05917739868164 N
index 37, latitude -48.757957458496094 N
index 49, latitude -30.455408096313477 N
index 50, latitude -28.716283798217773 N


In [13]:
def get_var_summer(year,runname,var,mo):
    tdir = '/gpfs/home/mep22dku/scratch/SOZONE/MO_RESULTS/EXTRACT/ncs/'
    if var == 'wspd':
        w = xr.open_dataset(f'{tdir}/scen_{runname}_{var}_1948_2100.nc')
    elif var == 'votemper':
        w = xr.open_dataset(f'{tdir}/TOM12_TJ_{runname}_{var}_L0_1948_2100.nc')
    elif var == 'EXP':
        w = xr.open_dataset(f'{tdir}/TOM12_TJ_{runname}_{var}_L10_1948_2100.nc')
    else:
        w = xr.open_dataset(f'{tdir}/TOM12_TJ_{runname}_{var}_1948_2100.nc')
    tdat = w[var].sel(time_counter=(w.time_counter.dt.month.isin([mo])&
                                      w.time_counter.dt.year.isin([year])))
    # print(np.shape(tdat.values))
    return tdat.values



print(np.shape(np.arange(1948,2101,1)))

runname = '1ASA'; var = 'pCO2'
yrstor = np.zeros([153,12,149,182])

for yr in range(1948,2101):
    for mo in range(1,13):
        td = get_var_summer(yr,runname,var,mo)
        yrstor[yr-1948,mo-1,:,:] = td[0,:,:]
        
print(np.shape(yrstor))

(153,)


KeyboardInterrupt: 

In [6]:
print(np.shape(yrstor))
print(bounds)
#1950 is index 3

#7 


(153, 12, 149, 182)
[ 0  7 16 22 28 32 37 40 44 47 50]


In [7]:
cflx_1ASA_blyat = np.zeros([1836,10])
for i in range(0,1836):
    tv = cflx_1ASA['cflx'].values
    for b in range(0,10):
        cflx_1ASA_blyat[i,b] = np.average(tv[i,bounds[b]:bounds[b+1],:], weights = csize[bounds[b]:bounds[b+1],:])


NameError: name 'cflx_1ASA' is not defined

In [ ]:
cflx_win = [cflx_winter_1ASA, cflx_winter_1BSA, cflx_winter_2ASA, cflx_winter_2BSA, cflx_winter_3ASA, cflx_winter_3BSA]

scens = ['1A', '1B', '2A', '2B', '3A', '3B']

fact = 0.45
fig, axs = plt.subplots(10,1, figsize=(14, 12), facecolor='w', edgecolor='k')
axs = axs.ravel()
fig.subplots_adjust(wspace=0, hspace=0.2)
for i in range(0,10):
    pos = 9-i
    for s in range(0,6):
        tb = cflx_sum[s]
        #axs[i].plot(myrs,tb[pos,:], alpha = 1, color = sc.scen[scens[s]]['color'], label = sc.scen[scens[s]]['name2'], linestyle = '-')
        tb = cflx_win[s]
        axs[i].plot(myrs,tb[pos,:], alpha = 1, color = sc.scen[scens[s]]['color'], label = sc.scen[scens[s]]['name2'], linestyle = '-')

    axs[4].legend(bbox_to_anchor = [1,1])
    axs[i].text(1950,0.9e-7,f'latitude band {pos}, {starts[pos]} $-$ {ends[pos]} $\degree$N')
    axs[i].set_ylim([-0.5e-7,1.9e-7])
    # st = cflx_1ASA['time_counter'].values[0]
    # en = cflx_1ASA['time_counter'].values[1835]
    axs[i].set_xlim([1948,2100])
    if i!= 9:
        axs[i].set_xticklabels([])
    axs[i].grid()
    axs[i].axhline(y=0)
plt.suptitle('latitude-band area-weighted winter cflx, yearly resolution, mol/m2/s, positive is sink-into-ocean', y = 0.9)

